In [1]:
import torch
from cephdataset import *

In [2]:
from train.pretrain import *
from train.pretrain import get_args
from mltool.universal_model_util import get_model_para_detail
import numpy as np
import os

In [3]:
args=get_args(config_path="checkpoints/WeathBench7066PatchDataset/PatchWrapper-AFNONet/time_step_2_pretrain-2D70N_every_1_step_random_dataset/11_14_22_22_24-seed_2022/config.json")

In [21]:
args.use_time_stamp=1
args.use_position_idx=1
args.wrapper_model = "POverLapTimePosBiasWrapper"
args.pretrain_weight =""
args.batch_size = 1
args.input_channel=77

In [22]:
args.pretrain_weight

''

In [23]:
args.use_wandb=0
args.gpu = args.local_rank = gpu  = local_rank = 0
##### parse args: dataset_kargs / model_kargs / train_kargs  ###########
args= parse_default_args(args)
SAVE_PATH = get_ckpt_path(args)
SAVE_PATH = "debug"
args.SAVE_PATH = str(SAVE_PATH)
#args.pretrain_weight = os.path.join(args.SAVE_PATH,'pretrain_latest.pt')
########## inital log ###################
logsys = create_logsys(args)


if args.distributed:
    if args.dist_url == "env://" and args.rank == -1:
        args.rank = int(os.environ["RANK"])
    if args.multiprocessing_distributed:
        # For multiprocessing distributed training, rank needs to be the
        # global rank among all the processes
        args.rank = args.rank * ngpus_per_node + local_rank
    logsys.info(f"start init_process_group,backend={args.dist_backend}, init_method={args.dist_url},world_size={args.world_size}, rank={args.rank}")
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,world_size=args.world_size, rank=args.rank)

model           = build_model(args)
#param_groups    = timm.optim.optim_factory.add_weight_decay(model, args.weight_decay)
optimizer,lr_scheduler,criterion = build_optimizer(args,model)
loss_scaler     = torch.cuda.amp.GradScaler(enabled=True)
logsys.info(f'use lr_scheduler:{lr_scheduler}')

args.pretrain_weight = args.pretrain_weight.strip()
logsys.info(f"loading weight from {args.pretrain_weight}")
start_epoch, start_step, min_loss = load_model(model.module if args.distributed else model, optimizer, lr_scheduler, loss_scaler, path=args.pretrain_weight, 
                    only_model= (args.mode=='fourcast') or (args.mode=='finetune' and not args.continue_train) ,loc = 'cuda:{}'.format(args.gpu))
if args.more_epoch_train:
    assert args.pretrain_weight
    print(f"detect more epoch training, we will do a copy processing for {args.pretrain_weight}")
    os.system(f'cp {args.pretrain_weight} {args.pretrain_weight}-epoch{start_epoch}')
logsys.info("done!")

2022-11-24 13:25:57,623 model args: img_size= (5, 5)
2022-11-24 13:25:57,623 model args: patch_size= 1
2022-11-24 13:25:57,644 use model ==> POverLapTimePosBiasWrapper
2022-11-24 13:25:57,645 Rank: 0, Local_rank: 0 | Number of Parameters: 3164703, Number of Buffers: 0, Size of Model: 12.0724 MB

2022-11-24 13:25:57,649 use lr_scheduler:None
2022-11-24 13:25:57,650 loading weight from 
2022-11-24 13:25:57,651 done!


log at debug
wandb id: 9034145ee668397cea5ca2cd4b077169
wandb is off, the recorder list is  ['tensorboard'], we pass wandb
dont find path, please check, we pass........


In [24]:
args.debug = 1

In [ ]:
train_dataset.vna

In [25]:

train_dataset, val_dataset, train_dataloader,val_dataloader = get_train_and_valid_dataset(args,
               train_dataset_tensor=None,train_record_load=None,
               valid_dataset_tensor=None,valid_record_load=None)
logsys.info(f"use dataset ==> {train_dataset.__class__.__name__}")
logsys.info(f"Start training for {args.epochs} epochs")
master_bar = logsys.create_master_bar(args.epochs)
accu_list = ['valid_loss']
metric_dict = logsys.initial_metric_dict(accu_list)

use dataset in datasets/weatherbench_6hour
load data from datasets/weatherbench_6hour/test.npy
use dataset in datasets/weatherbench_6hour
load data from datasets/weatherbench_6hour/test.npy


2022-11-24 13:26:00,972 use dataset ==> WeathBench7066PatchDataset
2022-11-24 13:26:00,973 Start training for 10000 epochs


In [17]:
model = model.float()

In [26]:
tensor, timestamp, postamp = next(iter(train_dataloader))[0]

In [11]:
postamp.shape

torch.Size([1])

In [27]:
x = model.image_to_patches(tensor,timestamp,postamp)

In [28]:
model(tensor.double(),timestamp.double(), postamp.double()).shape

RuntimeError: Input type (double) and bias type (float) should be the same

In [34]:
train_dataset.cross_sample

True

In [25]:
model(tensor[:1],timestamp[:1],postamp[:1])

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1792 but got size 1 for tensor number 1 in the list.

In [18]:
for epoch in master_bar:
    if epoch < start_epoch:continue
    if hasattr(model,'set_epoch'):model.set_epoch(epoch=epoch,epoch_total=args.epochs)
    if hasattr(model,'module') and hasattr(model.module,'set_epoch'):model.module.set_epoch(epoch=epoch,epoch_total=args.epochs)
    logsys.record('learning rate',optimizer.param_groups[0]['lr'],epoch, epoch_flag='epoch')
    train_loss = run_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,logsys,'train')


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 1835008 but got size 1024 for tensor number 1 in the list.

In [13]:
model.eval()
logsys.eval()
status     = 'test'
gpu        = dist.get_rank() if hasattr(model,'module') else 0
Fethcher   = Datafetcher
prefetcher = Fethcher(data_loader,next(model.parameters()).device)
batches = len(data_loader)
inter_b    = logsys.create_progress_bar(batches,unit=' img',unit_scale=data_loader.batch_size)
device = next(model.parameters()).device
data_cost = train_cost = rest_cost = 0
now = time.time()
model.clim = torch.Tensor(data_loader.dataset.clim_tensor).to(device)
fourcastresult={}
save_prediction_first_step = None#torch.zeros_like(data_loader.dataset.data)
save_prediction_final_step = None#torch.zeros_like(data_loader.dataset.data)
intervel = batches//100 + 1
with torch.no_grad():
    inter_b.lwrite("load everything, start_validating......", end="\r")

    while inter_b.update_step():
        #if inter_b.now>10:break
        data_cost += time.time() - now;now = time.time()
        step        = inter_b.now
        idxes,batch = prefetcher.next()
        batch       = make_data_regular(batch,half_model)
        # first sum should be (B, P, W, H )
        raise
        

NameError: name 'data_loader' is not defined

In [10]:
with torch.no_grad():
    accu_series=[]
    rmse_series=[]
    extra_info = {}
    time_step_1_mode=False
    clim = model.clim.detach().cpu()
    start = batch[0:model.history_length] # start must be a list
    for i in range(model.history_length,len(batch)):# i now is the target index
        print(batch[i].shape)
        print(start[0].shape)
        ltmv_pred, target, extra_loss, extra_info_from_model_list, start = once_forward(model,i,start,batch[i],dataset,time_step_1_mode)
        torch.cuda.empty_cache()
        print(ltmv_pred.shape)
        print(target.shape)

        for extra_info_from_model in extra_info_from_model_list:
            for key, val in extra_info_from_model.items():
                if i not in extra_info:extra_info[i] = {}
                if key not in extra_info[i]:extra_info[i][key] = []
                extra_info[i][key].append(val)


torch.Size([2, 70, 32, 64])
torch.Size([2, 70, 32, 64])
torch.Size([2, 70, 28, 64])
torch.Size([2, 70, 28, 64])
torch.Size([2, 70, 32, 64])
torch.Size([2, 70, 28, 64])
torch.Size([2, 70, 24, 64])
torch.Size([2, 70, 24, 64])
torch.Size([2, 70, 32, 64])
torch.Size([2, 70, 24, 64])
torch.Size([2, 70, 20, 64])
torch.Size([2, 70, 20, 64])
torch.Size([2, 70, 32, 64])
torch.Size([2, 70, 20, 64])
torch.Size([2, 70, 16, 64])
torch.Size([2, 70, 16, 64])
torch.Size([2, 70, 32, 64])
torch.Size([2, 70, 16, 64])
torch.Size([2, 70, 12, 64])
torch.Size([2, 70, 12, 64])
torch.Size([2, 70, 32, 64])
torch.Size([2, 70, 12, 64])
torch.Size([2, 70, 8, 64])
torch.Size([2, 70, 8, 64])
torch.Size([2, 70, 32, 64])
torch.Size([2, 70, 8, 64])
torch.Size([2, 70, 4, 64])
torch.Size([2, 70, 4, 64])
torch.Size([2, 70, 32, 64])
torch.Size([2, 70, 4, 64])


RuntimeError: cannot reshape tensor of 0 elements into shape [0, -1, 5, 5] because the unspecified dimension size -1 can be any value and is ambiguous

In [ ]:
        fourcastresult,extra_info = run_one_fourcast_iter(model, batch, idxes, fourcastresult,data_loader.dataset,
                                     save_prediction_first_step=save_prediction_first_step,save_prediction_final_step=save_prediction_final_step)
        train_cost += time.time() - now;now = time.time()
        start = batch[0]
        for _ in range(random_repeat):
            batch[0] = start*(1 + torch.randn_like(start)*0.05)
            fourcastresult,extra_info = run_one_fourcast_iter(model, batch, idxes, fourcastresult,data_loader.dataset)
        rest_cost += time.time() - now;now = time.time()
        if (step+1) % intervel==0 or step==0:
            for idx, val_pool in extra_info.items():
                info_pool={}
                for key, val in val_pool.items():
                    logsys.record(f'test_{key}_each_fourcast_step', np.mean(val), idx, epoch_flag = 'time_step')
                    info_pool[f'test_{key}_each_fourcast_step'] = np.mean(val)
            outstring=(f"epoch:fourcast iter:[{step:5d}]/[{len(data_loader)}] GPU:[{gpu}] cost:[Date]:{data_cost/intervel:.1e} [Train]:{train_cost/intervel:.1e} [Rest]:{rest_cost/intervel:.1e}")
            inter_b.lwrite(outstring, end="\r")
if save_prediction_first_step is not None:torch.save(save_prediction_first_step,os.path.join(logsys.ckpt_root,'save_prediction_first_step')) 
if save_prediction_final_step is not None:torch.save(save_prediction_final_step,os.path.join(logsys.ckpt_root,'save_prediction_final_step')) 

In [ ]:
train_dataset_tensor=None;
train_record_load=None;
valid_dataset_tensor=None;
valid_record_load=None
args.debug = True
args.valid_batch_size=3
train_dataset, val_dataset, train_dataloader,val_dataloader = get_train_and_valid_dataset(args,
               train_dataset_tensor=train_dataset_tensor,train_record_load=train_record_load,
               valid_dataset_tensor=valid_dataset_tensor,valid_record_load=valid_record_load)

In [ ]:
train_dataset.load_otensor(1).shape

In [ ]:
train_dataset[0][0].shape

In [ ]:
train_dataset_tensor=None;
train_record_load=None;
valid_dataset_tensor=None;
valid_record_load=None

In [ ]:
# =======================> start training <==========================
print(f"entering {args.mode} training in {next(model.parameters()).device}")
now_best_path = SAVE_PATH / 'backbone.best.pt'
latest_ckpt_p = SAVE_PATH / 'pretrain_latest.pt'


train_dataset, val_dataset, train_dataloader,val_dataloader = get_train_and_valid_dataset(args,
               train_dataset_tensor=train_dataset_tensor,train_record_load=train_record_load,
               valid_dataset_tensor=valid_dataset_tensor,valid_record_load=valid_record_load)
logsys.info(f"use dataset ==> {train_dataset.__class__.__name__}")
logsys.info(f"Start training for {args.epochs} epochs")
metric_list = ['loss']
master_bar        = logsys.create_master_bar(args.epochs)
master_bar.set_multiply_graph(figsize=(9,3),engine=[['plot','plot']],labels=[metric_list])
for epoch in master_bar:
    if epoch < start_epoch:continue
    if hasattr(model,'set_epoch'):model.set_epoch(epoch=epoch,epoch_total=args.epochs)
    if hasattr(model,'module') and hasattr(model.module,'set_epoch'):model.module.set_epoch(epoch=epoch,epoch_total=args.epochs)
    logsys.record('learning rate',optimizer.param_groups[0]['lr'],epoch)
    train_loss = run_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,logsys,'train')
    if (not args.more_epoch_train) and (lr_scheduler is not None):lr_scheduler.step(epoch)
    #torch.cuda.empty_cache()
    #train_loss = single_step_evaluate(train_dataloader, model, criterion,epoch,logsys,status='train') if 'small' in args.train_set else -1
    val_loss   = run_one_epoch(epoch, start_step, model, criterion, val_dataloader, optimizer, loss_scaler,logsys,'valid')

    if (not args.distributed) or (args.rank == 0 and local_rank == 0) :
        logsys.info(f"Epoch {epoch} | Train loss: {train_loss:.6f}, Val loss: {val_loss:.6f}")
        logsys.record('train', train_loss, epoch)
        logsys.record('valid', val_loss, epoch)
        if use_wandb:wandb.log({"epoch":epoch,'train':train_loss,'valid':val_loss})
        if val_loss < min_loss:
            min_loss = val_loss
            if epoch > args.epochs//10:
                logsys.info(f"saving best model ....")
                save_model(model, path=now_best_path, only_model=True)
                logsys.info(f"done;")
            #if last_best_path is not None:os.system(f"rm {last_best_path}")
            #last_best_path= now_best_path
            logsys.info(f"The best accu is {val_loss}")
        logsys.record('best_loss', min_loss, epoch)
        update_experiment_info(experiment_hub_path,epoch,args)
        if epoch>args.save_warm_up:
            logsys.info(f"saving latest model ....")
            save_model(model, epoch+1, 0, optimizer, lr_scheduler, loss_scaler, min_loss, latest_ckpt_p)
            logsys.info(f"done ....")

if os.path.exists(now_best_path) and args.do_final_fourcast:
    logsys.info(f"we finish training, then start test on the best checkpoint {now_best_path}")
    start_epoch, start_step, min_loss = load_model(model.module if args.distributed else model, path=now_best_path, only_model=True)
    run_fourcast(args, model,logsys)
if use_wandb:wandb.finish()

In [ ]:
if local_rank == 0:
    print(f"Start training for {args.epochs} epochs")

master_bar        = logsys.create_master_bar(args.epochs)
last_best_path = None
for epoch in master_bar:
    if epoch < start_epoch:continue
    train_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,lr_scheduler, min_loss,logsys)
    lr_scheduler.step(epoch)
    #torch.cuda.empty_cache()
    train_loss = single_step_evaluate(train_dataloader, model, criterion,epoch,logsys)
    #train_loss = -1
    val_loss   = single_step_evaluate(val_dataloader, model, criterion,epoch,logsys)

    if rank == 0 and local_rank == 0:
        print(f"Epoch {epoch} | Train loss: {train_loss:.6f}, Val loss: {val_loss:.6f}")
        logsys.record('train', train_loss, epoch)
        logsys.record('valid', val_loss, epoch)
        if val_loss < min_loss:
            min_loss = val_loss
            print(f"saving best model ....")
            now_best_path = SAVE_PATH / f'backbone.best.pt'
            if epoch>args.save_warm_up:save_model(model, path=now_best_path, only_model=True)
            #if last_best_path is not None:os.system(f"rm {last_best_path}")
            #last_best_path= now_best_path
            print(f"done; the best accu is {val_loss}")
        logsys.record('best_loss', min_loss, epoch)
        

In [ ]:
if __name__ == '__main__':
    args = get_args()
    ngpus = ngpus_per_node = torch.cuda.device_count()
    args.world_size = -1
    args.dist_file  = None
    args.rank       = 0
    args.dist_backend = "nccl"
    args.multiprocessing_distributed = ngpus>1
    if not hasattr(args,'train_set'):args.train_set='large'
    ip = os.environ.get("MASTER_ADDR", "127.0.0.1")
    port = os.environ.get("MASTER_PORT", "54247")
    hosts = int(os.environ.get("WORLD_SIZE", "1"))  # number of nodes
    rank = int(os.environ.get("RANK", "0"))  # node id
    gpus = torch.cuda.device_count()  # gpus per node
    args.dist_url = f"tcp://{ip}:{port}"
    if args.world_size == -1 and "SLURM_NPROCS" in os.environ:
        args.world_size = int(os.environ["SLURM_NPROCS"])
        args.rank       = int(os.environ["SLURM_PROCID"])
        jobid           = os.environ["SLURM_JOBID"]

        hostfile        = "dist_url." + jobid  + ".txt"
        if args.dist_file is not None:
            args.dist_url = "file://{}.{}".format(os.path.realpath(args.dist_file), jobid)
        elif args.rank == 0:
            import socket
            ip = socket.gethostbyname(socket.gethostname())
            port = find_free_port()
            args.dist_url = "tcp://{}:{}".format(ip, port)
            #with open(hostfile, "w") as f:f.write(args.dist_url)
        else:
            import os
            import time
            while not os.path.exists(hostfile):
                time.sleep(1)
            with open(hostfile, "r") as f:
                args.dist_url = f.read()
        print("dist-url:{} at PROCID {} / {}".format(args.dist_url, args.rank, args.world_size))
    else:
        args.world_size = 1
    args.distributed = args.world_size > 1 or args.multiprocessing_distributed
    train_dataset_tensor=valid_dataset_tensor=None

    print("======== loading data ==========")
    if 'small' in args.train_set:
        if not args.fourcast:
            train_dataset_tensor = load_small_dataset_in_memory('train').share_memory_()
            valid_dataset_tensor = load_small_dataset_in_memory('valid').share_memory_()
        else:
            train_dataset_tensor = load_small_dataset_in_memory('test').share_memory_()
            valid_dataset_tensor = None
    else:
        if args.fourcast:
            train_dataset_tensor = load_test_dataset_in_memory(years=[2018],root="/nvme/zhangtianning/datasets/ERA5").share_memory_()
            valid_dataset_tensor = None
    print("=======done==========")
    print(train_dataset_tensor.shape)
    if args.multiprocessing_distributed:
        args.world_size = ngpus_per_node * args.world_size
        torch.multiprocessing.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args,train_dataset_tensor,valid_dataset_tensor))
    else:
        main_worker(0, ngpus_per_node, args,train_dataset_tensor,valid_dataset_tensor)